In [1]:
import os

import xarray as xr
import matplotlib.pyplot as plt
import ncar_jobqueue

import data_catalog
import esmlab_wrap
import tseries_mod

In [2]:
data_catalog.set_catalog('experiments')

active catalog: experiments


In [3]:
clobber = os.environ['CLOBBER'] == 'True' if 'CLOBBER' in os.environ else False
cluster = ncar_jobqueue.NCARCluster() if clobber else None

In [4]:
varnames = ['FG_CO2', 'DIC_vertint']

In [5]:
experiment = 'piControl'
piControl = tseries_mod.tseries_get_vars(varnames, 'ocn', experiment, cluster_in=cluster)
piControl_ann = esmlab_wrap.compute_ann_mean(piControl)
# print(piControl_ann)

In [6]:
experiment = 'piControl-waccm'
piControl_waccm = tseries_mod.tseries_get_vars(varnames, 'ocn', experiment, cluster_in=cluster)
piControl_waccm_ann = esmlab_wrap.compute_ann_mean(piControl_waccm)
# print(piControl_waccm_ann)

In [7]:
experiment = 'esm-piControl'
esm_piControl = tseries_mod.tseries_get_vars(varnames, 'ocn', experiment, cluster_in=cluster)
esm_piControl_ann = esmlab_wrap.compute_ann_mean(esm_piControl)
# print(esm_piControl_ann)

In [8]:
if clobber:
    cluster.close()

In [9]:
print(piControl_ann.mean('time').sel(region='SouOce'))
print(piControl_waccm_ann.mean('time').sel(region='SouOce'))
print(esm_piControl_ann.mean('time').sel(region='SouOce'))

<xarray.Dataset>
Dimensions:      ()
Coordinates:
    region       <U6 'SouOce'
Data variables:
    FG_CO2       float64 0.2243
    DIC_vertint  float64 1.126e+04
Attributes:
    history:  \n2019-11-09 15:03:39.717028 esmlab.resample(<DATASET>, freq="a...
<xarray.Dataset>
Dimensions:      ()
Coordinates:
    region       <U6 'SouOce'
Data variables:
    FG_CO2       float64 0.2144
    DIC_vertint  float64 1.124e+04
Attributes:
    history:  \n2019-11-09 15:03:57.713515 esmlab.resample(<DATASET>, freq="a...
<xarray.Dataset>
Dimensions:      ()
Coordinates:
    region       <U6 'SouOce'
Data variables:
    FG_CO2       float64 0.2352
    DIC_vertint  float64 1.128e+04
Attributes:
    history:  \n2019-11-09 15:04:17.046833 esmlab.resample(<DATASET>, freq="a...
